In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics

import holisticai
from holisticai.bias.mitigation.postprocessing import CalibratedEqualizedOdds, RejectOptionClassification

<h4> Data Loading and Baseline Modeling

Data Loading

In [87]:
import os
os.getcwd()

'c:\\Users\\athar\\OneDrive\\Desktop\\Rutgers\\Ethical Stat Learning\\Project\\compas-fairness-privacy-tradeoff-analysis\\Fairness'

In [88]:
baseline_preprocessed_df = pd.read_csv("C:/Users/athar/OneDrive/Desktop/Rutgers/Ethical Stat Learning/Project/data/data_preprocessed_baseline.csv",index_col=0)
baseline_preprocessed_df = baseline_preprocessed_df.drop(['id','age_cat'],axis=1)
#baseline_preprocessed_df['two_year_recid'] = np.where(baseline_preprocessed_df['two_year_recid'] == 1, 0, 1)

In [89]:
protected_variables = ["sex", "African-American_race"]
output_variable = ["two_year_recid"]

group = ["African-American_race"]
group_a = baseline_preprocessed_df["African-American_race"] == 1
group_b = baseline_preprocessed_df["African-American_race"] == 0

In [90]:
y = baseline_preprocessed_df[output_variable]
X = pd.get_dummies(baseline_preprocessed_df.drop(output_variable, axis=1))
data_ = [X, y, group_a, group_b]

In [91]:
dataset = train_test_split(*data_, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]

In [92]:
# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [93]:
#implement logistic regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [94]:
# pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ])

In [95]:
# train
X_train, y_train, group_a_train, group_b_train = train_data
pipeline.fit(X_train, y_train)

# predict train set
y_pred_train = pipeline.predict(X_train)
y_proba_train = pipeline.predict_proba(X_train)

# predict test set
X_test, y_test, group_a_test, group_b_test = test_data
y_pred_test = pipeline.predict(X_test)
y_proba_test = pipeline.predict_proba(X_test)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<h3>Equality of opportunity

For each group, of the individuals who should have a positive label (ground truth), their probability of having a positive label is the same.
	
Often does not make a difference to the FPR and FRN rates, but improves equalized odds. Therefore we will not be using this technique for our purpose

<h3>Calibrated Equalized Odds

seeks to ensure both calibration and equalized odds. 
Calibration : Refers to the property that the model's predicted probabilities accurately reflect the true probabilities of the target variable. Equalized odds : refers to the property that the model has the same true positive rate (TPR) and false positive rate (FPR) for all protected groups.
	
	
In other words, a calibrated equalized odds model should make predictions that are accurate and fair, regardless of the protected group of an individual. This is a challenging task, as it requires the model to balance two competing objectives: calibration and fairness.


In [96]:
# initialize object
ceo = CalibratedEqualizedOdds(cost_constraint="fpr")

In [97]:
# fit it
ceo.fit(y_train, y_proba_train, group_a_train, group_b_train)

In [98]:
# transform it
d = ceo.transform(y_test, y_proba_test, group_a_test, group_b_test, 0.65)

In [99]:
# new predictions
y_pred_ceo = d['y_pred']

In [100]:
# efficacy
metrics_dataframe(y_pred_ceo, y_test)

,Value
Metric,
Accuracy,0.654836
Balanced accuracy,0.639448
Precision,0.681716
Recall,0.459665
F1-Score,0.549091


In [101]:
from holisticai.bias.metrics import classification_bias_metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred_ceo, y_test, metric_type='both')

c:\Users\athar\anaconda3\lib\site-packages\holisticai\bias\metrics\_classification.py:172: RuntimeWarning: divide by zero encountered in scalar divide
  return sr_a / sr_b
c:\Users\athar\anaconda3\lib\site-packages\holisticai\bias\metrics\_classification.py:223: RuntimeWarning: divide by zero encountered in scalar divide
  return min(sr_a / sr_b, sr_b / sr_a)


,Value,Reference
Metric,,
Statistical Parity,0.583663,0
Disparate Impact,inf,1
Four Fifths Rule,0.000000,1
Cohen D,1.629107,0
2SD Rule,23.918339,0
Equality of Opportunity Difference,0.760705,0
False Positive Rate Difference,0.389503,0
Average Odds Difference,0.575104,0
Accuracy Difference,0.072545,0


<h3>Reject Option Classification

Reject Option Classification (ROC) is a classification technique in machine learning that allows the classifier to abstain from making a prediction when the confidence in its prediction is low. This is in contrast to traditional classification, where the classifier is forced to make a prediction for every input instance, even if the classifier is uncertain about the true label.


ROC is particularly useful in situations where making an incorrect decision can have significant consequences, such as in medical diagnosis or fraud detection. By allowing the classifier to abstain from making a prediction when it is uncertain, ROC can help to reduce the risk of making errors.

In [110]:
baseline_preprocessed_df['two_year_recid'] = np.where(baseline_preprocessed_df['two_year_recid'] == 1, 0, 1)

y = baseline_preprocessed_df[output_variable]
X = pd.get_dummies(baseline_preprocessed_df.drop(output_variable, axis=1))
data_ = [X, y, group_a, group_b]

dataset = train_test_split(*data_, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]


# train
X_train, y_train, group_a_train, group_b_train = train_data
pipeline.fit(X_train, y_train)

# predict train set
y_pred_train = pipeline.predict(X_train)
y_proba_train = pipeline.predict_proba(X_train)

# predict test set
X_test, y_test, group_a_test, group_b_test = test_data
y_pred_test = pipeline.predict(X_test)
y_proba_test = pipeline.predict_proba(X_test)


c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [111]:
# initialize
roc = RejectOptionClassification(metric_name="Statistical parity difference")

In [112]:
# fit it
roc.fit(y_train, y_proba_train, group_a_train, group_b_train)

In [113]:
# transform it
d = roc.transform(y_test, y_proba_test, group_a_test, group_b_test)
d

{'y_pred': array([ True, False,  True, ...,  True,  True, False]),
 'y_score': array([0.75048265, 0.55095153, 0.68877118, ..., 0.67994792, 0.64927536,
        0.61921092])}

In [114]:
# new predictions
y_pred = d['y_pred']
y_pred

array([ True, False,  True, ...,  True,  True, False])

In [115]:
# efficacy
metrics_dataframe(y_pred, y_test)

,Value
Metric,
Accuracy,0.677801
Balanced accuracy,0.679840
Precision,0.728291
Recall,0.659062
F1-Score,0.691949


In [116]:
# bias metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.027479,0
Disparate Impact,1.056951,1
Four Fifths Rule,0.946117,1
Cohen D,0.054981,0
2SD Rule,1.040637,0
Equality of Opportunity Difference,0.097446,0
False Positive Rate Difference,0.037807,0
Average Odds Difference,0.067626,0
Accuracy Difference,0.041760,0


In [103]:
y = baseline_preprocessed_df[output_variable]

X = pd.get_dummies(baseline_preprocessed_df.drop(output_variable, axis=1))
data_ = [X, y, group_a, group_b]